In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key: ")

In [ ]:
!mkdir static/
!mkdir static/training_data
!curl https://python.langchain.com/docs/tutorials/rag/ -o static/training_data/langchain_rag_tutorial.html

mkdir: static/training_data: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  340k  100  340k    0     0  2270k      0 --:--:-- --:--:-- --:--:-- 2285k


In [4]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import BSHTMLLoader

path = "static/training_data/"
text_loader = DirectoryLoader(path, glob="*.html", loader_cls=BSHTMLLoader)
docs = text_loader.load()
len(docs)

1

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 750,
    chunk_overlap  = 20,
    length_function = len
)
training_documents = text_splitter.split_documents(text_loader.load())
len(training_documents)

81

In [6]:
import uuid

id_set = set()

for document in training_documents:
  id = str(uuid.uuid4())
  while id in id_set:
    id = uuid.uuid4()
  id_set.add(id)
  document.metadata["id"] = id

In [7]:
# break up training documents into training, validation, and test sets
import random

# set seed for reproducibility
random.seed(42)

random.shuffle(training_documents)

training_split_documents = training_documents[:int(0.8 * len(training_documents))]
val_split_documents = training_documents[int(0.8 * len(training_documents)):int(0.9 * len(training_documents))]
test_split_documents = training_documents[int(0.9 * len(training_documents)):]

In [8]:
from langchain_openai import ChatOpenAI

qa_chat_model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0
)

from langchain_core.prompts import ChatPromptTemplate

qa_prompt = """\
Given the following context, you must generate questions based on only the provided context.

You are to generate {n_questions} questions which should be provided in the following format:

1. QUESTION #1
2. QUESTION #2
...

Context:
{context}
"""

qa_prompt_template = ChatPromptTemplate.from_template(qa_prompt)
question_generation_chain = qa_prompt_template | qa_chat_model

In [9]:
import tqdm

async def create_questions(documents, n_questions):
  questions = {}
  contexts = {}
  for document in documents:
    question = await question_generation_chain.ainvoke({"context": document.page_content, "n_questions": n_questions})
    questions[document.metadata["id"]] = question
    contexts[document.metadata["id"]] = [document.metadata["id"]]
  return questions, contexts

In [10]:
training_questions, training_relevant_contexts = await create_questions(training_split_documents, 2)
val_questions, val_relevant_contexts = await create_questions(val_split_documents, 2)
test_questions, test_relevant_contexts = await create_questions(test_split_documents, 2)

In [11]:
import json

training_corpus = {train_item.metadata["id"] : train_item.page_content for train_item in training_split_documents}

# Convert AIMessage objects to their string content
training_questions_serializable = {k: v.content for k, v in training_questions.items()}

train_dataset = {
    "questions": training_questions_serializable,
    "relevant_contexts": training_relevant_contexts,
    "corpus": training_corpus
}

with open("static/training_data/training_dataset.jsonl", "w") as f:
    json.dump(train_dataset, f)

val_corpus = {val_item.metadata["id"] : val_item.page_content for val_item in val_split_documents}

# Convert AIMessage objects to their string content
val_questions_serializable = {k: v.content for k, v in val_questions.items()}

val_dataset = {
    "questions": val_questions_serializable,
    "relevant_contexts": val_relevant_contexts,
    "corpus": val_corpus
}

with open("static/training_data/val_dataset.jsonl", "w") as f:
    json.dump(val_dataset, f)

test_corpus = {test_item.metadata["id"] : test_item.page_content for test_item in test_split_documents}

# Convert AIMessage objects to their string content
test_questions_serializable = {k: v.content for k, v in test_questions.items()}

test_dataset = {
    "questions": test_questions_serializable,
    "relevant_contexts": test_relevant_contexts,
    "corpus": test_corpus
}

with open("static/training_data/test_dataset.jsonl", "w") as f:
    json.dump(test_dataset, f)

In [12]:
from sentence_transformers import SentenceTransformer

model_id = "Snowflake/snowflake-arctic-embed-l"
model = SentenceTransformer(model_id)

/Users/ryanrodriguez/src/Simplify/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sentence_transformers import InputExample

corpus = train_dataset['corpus']
queries = train_dataset['questions']
relevant_docs = train_dataset['relevant_contexts']

examples = []
for query_id, query in queries.items():
    doc_id = relevant_docs[query_id][0]
    text = corpus[doc_id]
    example = InputExample(texts=[query, text])
    examples.append(example)

BATCH_SIZE = 16

loader = DataLoader(
    examples,
    batch_size=BATCH_SIZE,
)

from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss

matryoshka_dimensions = [768, 512, 256, 128, 64]
inner_train_loss = MultipleNegativesRankingLoss(model)
train_loss = MatryoshkaLoss(
    model, inner_train_loss, matryoshka_dims=matryoshka_dimensions
)

In [14]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = val_dataset['corpus']
queries = val_dataset['questions']
relevant_docs = val_dataset['relevant_contexts']

evaluator = InformationRetrievalEvaluator(queries, corpus, relevant_docs)

In [15]:
EPOCHS = 10

In [16]:
import wandb
wandb.init(mode="disabled")

In [17]:
warmup_steps = int(len(loader) * EPOCHS * 0.1)

model.fit(
    train_objectives=[(loader, train_loss)],
    epochs=EPOCHS,
    warmup_steps=warmup_steps,
    output_path='finetuned_arctic_ft',
    show_progress_bar=True,
    evaluator=evaluator,
    evaluation_steps=50
)

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
hf_username = "Rsr2425"
model.push_to_hub(f"{hf_username}/simplify-embeddings")